##LIBRERIAS

In [4]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
##FOR COLAB
# from google.colab import drive 
# drive.mount('/content/drive')

##LIBRERIAS MODELO

In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import joblib

##SUBIR ARCHIVO

In [6]:
# Ruta a la carpeta donde se ubica la data.
folder_path = 'Data/Cuestionario_Clima_2023_Co_Depurado (1).xlsx'
df = pd.read_excel(folder_path,header=1)
df.head()

,Marca temporal,Código de la institución,1. Nombre de la institución,2. Tipo de Institución,3. Denominación de los servicios instituciones en salud,4. Área de trabajo.,5. Edad (años cumplidos),6. Sexo,7. Estado civil,8. Escolaridad,...,Equidad,Compromiso con la productividad,Compatibilidad de intereses,Intercambio de información,Involucración al cambio,LIDERAZGO,MOTIVACIÓN,RECIPROCIDAD,PARTICIPACIÓN,Id
0,2018-12-17 09:16:57.471,76001CML,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,30,Mujer,Casado,Técnico,...,3,5,4,1,3,3.75,4.00,4.00,3.25,0
1,2018-12-21 15:26:21.193,76001CML,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,27,Hombre,Soltero,Universitaria,...,3,5,5,4,4,4.75,4.75,3.50,4.50,1
2,2018-12-21 15:33:46.427,76001CML,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,21,Mujer,Soltero,Técnico,...,2,5,5,4,4,4.50,4.50,3.25,4.50,2
3,2018-12-21 15:46:42.083,76001CML,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,22,Mujer,Soltero,Universitaria,...,4,5,4,3,3,4.50,4.50,3.75,3.75,3
4,2018-12-21 15:59:00.247,76001CML,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,43,Mujer,Soltero,Secundaria,...,3,5,5,4,4,4.50,4.25,3.50,4.50,4


##PREPROCESO

In [7]:
CualitativesValues=[
#'Código de la institución  '
'1. Nombre de la institución '
,'2. Tipo de Institución '
,'3. Denominación de los servicios instituciones en salud '
,'4. Área de trabajo.'
,'5. Edad (años cumplidos)'
,'6. Sexo','7. Estado civil'
,'8. Escolaridad'
,'9. Termino de contratación'
,'10. Tipo de contrato',
'11. Horas laboradas cada día en la institución '
,'12. Tiempo Laborado en la institución'
,'13. Tiempo Laborado en su actual área o servicio'
,'14. Tiempo laborado en su actual cargo']

CualitativesValuesOneHot=[
'1. Nombre de la institución '
,'2. Tipo de Institución '
,'4. Área de trabajo.'
,'6. Sexo'
,'7. Estado civil'
,'9. Termino de contratación'
,'10. Tipo de contrato',
]
CualitativesValuesOneHotPrefix=[
'Nombre de la institución '
,'Tipo de Institución '
,'Área de trabajo.'
,'Sexo'
,'Estado civil'
,'Termino de contratación'
,'Tipo de contrato',
]

CualitativesValuesLabelEncoding=[
'3. Denominación de los servicios instituciones en salud ',
'8. Escolaridad',
'5. Edad (años cumplidos)',
'11. Horas laboradas cada día en la institución ',
'12. Tiempo Laborado en la institución',
'13. Tiempo Laborado en su actual área o servicio',
'14. Tiempo laborado en su actual cargo',
]

seleccionamos solo los atributos de entrada

In [8]:
df_filter_cualidades = df.iloc[:,2:16] ##SELECCIONAMOS TODAS LAS FILAS ## ELIMINAMOS LA FECHA Y EL CODIGO
df_filter_cualidades

,1. Nombre de la institución,2. Tipo de Institución,3. Denominación de los servicios instituciones en salud,4. Área de trabajo.,5. Edad (años cumplidos),6. Sexo,7. Estado civil,8. Escolaridad,9. Termino de contratación,10. Tipo de contrato,11. Horas laboradas cada día en la institución,12. Tiempo Laborado en la institución,13. Tiempo Laborado en su actual área o servicio,14. Tiempo laborado en su actual cargo
0,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,30,Mujer,Casado,Técnico,Término indefinido,Prestación de servicios,1–4 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
1,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,27,Hombre,Soltero,Universitaria,Término indefinido,Por la institución,9-12 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
2,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,21,Mujer,Soltero,Técnico,Término indefinido,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
3,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,22,Mujer,Soltero,Universitaria,Término indefinido,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
4,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,43,Mujer,Soltero,Secundaria,Término fijo,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial ambulatorio,32,Hombre,Unión Libre,Técnico,Prestación de servicios,Por Cooperativa o asociación,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año
2012,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial hospitalario,43,Mujer,Soltero,Universitaria,Término fijo,Por la institución,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año
2013,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Apoyo operativo,29,Mujer,Soltero,Técnico,Término fijo,Por Cooperativa o asociación,5-8 horas,De 1 a 5 años,Menos de 1 año,Menos de 1 año
2014,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial hospitalario,37,Hombre,Unión Libre,Técnico,Término fijo,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años


##ONEHOT

In [9]:
# Aplica la codificación one-hot

# Aplica one-hot encoding a las columnas 'genero', 'tipo_contrato' y 'estado_civil'
df_encoded = pd.get_dummies(df_filter_cualidades[CualitativesValuesOneHot], prefix=CualitativesValuesOneHotPrefix)
Arreglo = CualitativesValuesOneHot.copy()
# Concatena el DataFrame codificado con el DataFrame original
df_OneHot = pd.concat([df_filter_cualidades, df_encoded], axis=1)
df_OneHot.drop(Arreglo, axis=1, inplace=True)
df_OneHot

,3. Denominación de los servicios instituciones en salud,5. Edad (años cumplidos),8. Escolaridad,11. Horas laboradas cada día en la institución,12. Tiempo Laborado en la institución,13. Tiempo Laborado en su actual área o servicio,14. Tiempo laborado en su actual cargo,Nombre de la institución _CENTRO MEDICO POR SALUD,Nombre de la institución _CLINICA MARIANGEL TULUA,Nombre de la institución _CLINICA SAN JUAN DE DIOS DE MANIZALES,...,Estado civil_Unión Libre,Estado civil_Viudo,Termino de contratación_Prestación de servicios,Termino de contratación_Término fijo,Termino de contratación_Término indefinido,Tipo de contrato_Carrera administrativa,Tipo de contrato_Otra,Tipo de contrato_Por Cooperativa o asociación,Tipo de contrato_Por la institución,Tipo de contrato_Prestación de servicios
0,Baja Complejidad,30,Técnico,1–4 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,Baja Complejidad,27,Universitaria,9-12 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,Baja Complejidad,21,Técnico,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,Baja Complejidad,22,Universitaria,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,Baja Complejidad,43,Secundaria,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,Mediana complejidad,32,Técnico,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año,0,0,0,...,1,0,1,0,0,0,0,1,0,0
2012,Mediana complejidad,43,Universitaria,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2013,Mediana complejidad,29,Técnico,5-8 horas,De 1 a 5 años,Menos de 1 año,Menos de 1 año,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2014,Mediana complejidad,37,Técnico,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,0,0,0,...,1,0,0,1,0,0,0,0,1,0


##ORDINAL ENCONDING

In [10]:
"""
APLICAMOS LABEL ENCODING A LAS VARIABLES QUE LO AMERITAN
"""
CualitativesValuesLabelEncoding=[
'3. Denominación de los servicios instituciones en salud ',
'8. Escolaridad',
'11. Horas laboradas cada día en la institución ',
'12. Tiempo Laborado en la institución',
'13. Tiempo Laborado en su actual área o servicio',
'14. Tiempo laborado en su actual cargo',
]

df_OneHot['Denominación de los servicios instituciones en salud_encoded'] = df_OneHot['3. Denominación de los servicios instituciones en salud '].map({'Alta complejidad': 2, 'Baja Complejidad': 0, 'Mediana complejidad':1})
df_OneHot['Escolaridad_encoded'] = df_OneHot['8. Escolaridad'].map({'Primaria': 0, 'Secundaria': 1,'Técnico':2,'Universitaria':3,'Posgrado':4})
df_OneHot['Horas laboradas_encoded'] = df_OneHot['11. Horas laboradas cada día en la institución '].map({'1–4 horas':0, '5-8 horas':1,'9-12 horas':2,'Más de 12 horas':3})
df_OneHot['Tiempo_institución_encoded'] = df_OneHot['12. Tiempo Laborado en la institución'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_OneHot['Tiempo_servicio_encoded'] = df_OneHot['13. Tiempo Laborado en su actual área o servicio'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_OneHot['Tiempo_actual_encoded'] = df_OneHot['14. Tiempo laborado en su actual cargo'].map({'De 1 a 5 años': 1, 'De 6 a 10 años': 2 ,'Menos de 1 año':0,'11 años o más':3})
df_OneHot.drop(CualitativesValuesLabelEncoding, axis=1, inplace=True)
df_OneHot

,5. Edad (años cumplidos),Nombre de la institución _CENTRO MEDICO POR SALUD,Nombre de la institución _CLINICA MARIANGEL TULUA,Nombre de la institución _CLINICA SAN JUAN DE DIOS DE MANIZALES,Nombre de la institución _CLINICA VERSALLES,Nombre de la institución _Centro médico laboral,Nombre de la institución _Christus sinergia Tuluá,Nombre de la institución _ESE CAMILO TRUJILLO SILVA,Nombre de la institución _ESE HOSPITAL LOCAL DE CANDELARIA,Nombre de la institución _ESE HOSPITAL MUNICIPAL ALGECIRAS,...,Tipo de contrato_Otra,Tipo de contrato_Por Cooperativa o asociación,Tipo de contrato_Por la institución,Tipo de contrato_Prestación de servicios,Denominación de los servicios instituciones en salud_encoded,Escolaridad_encoded,Horas laboradas_encoded,Tiempo_institución_encoded,Tiempo_servicio_encoded,Tiempo_actual_encoded
0,30,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,2,0,1,1,1
1,27,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,2,1,1,1
2,21,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,2,1,1,1,1
3,22,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,3,1,1,1,1
4,43,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,32,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,2,1,0,0,0
2012,43,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,3,1,0,0,0
2013,29,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,2,1,1,0,0
2014,37,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,2,1,1,1,1


##DEFINIMOS LOS RESULTADOS DEL EXAMEN

In [11]:
##DEFINIMOS LAS SALIDAS
df_filter_values = df.iloc[:,-5:-1]
df_filter_values

,LIDERAZGO,MOTIVACIÓN,RECIPROCIDAD,PARTICIPACIÓN
0,3.75,4.00,4.00,3.25
1,4.75,4.75,3.50,4.50
2,4.50,4.50,3.25,4.50
3,4.50,4.50,3.75,3.75
4,4.50,4.25,3.50,4.50
...,...,...,...,...
2011,3.25,3.25,3.50,3.25
2012,1.75,3.75,3.50,1.75
2013,3.50,3.75,3.75,3.25
2014,2.75,4.00,3.25,2.50


##GENERAMOS EL ARREGLO COMPLETO PARA EL CLUSTER

In [12]:
df_OneHot_results=pd.concat([df_OneHot, df_filter_values], axis=1)
df_OneHot_results

,5. Edad (años cumplidos),Nombre de la institución _CENTRO MEDICO POR SALUD,Nombre de la institución _CLINICA MARIANGEL TULUA,Nombre de la institución _CLINICA SAN JUAN DE DIOS DE MANIZALES,Nombre de la institución _CLINICA VERSALLES,Nombre de la institución _Centro médico laboral,Nombre de la institución _Christus sinergia Tuluá,Nombre de la institución _ESE CAMILO TRUJILLO SILVA,Nombre de la institución _ESE HOSPITAL LOCAL DE CANDELARIA,Nombre de la institución _ESE HOSPITAL MUNICIPAL ALGECIRAS,...,Denominación de los servicios instituciones en salud_encoded,Escolaridad_encoded,Horas laboradas_encoded,Tiempo_institución_encoded,Tiempo_servicio_encoded,Tiempo_actual_encoded,LIDERAZGO,MOTIVACIÓN,RECIPROCIDAD,PARTICIPACIÓN
0,30,0,0,0,0,1,0,0,0,0,...,0,2,0,1,1,1,3.75,4.00,4.00,3.25
1,27,0,0,0,0,1,0,0,0,0,...,0,3,2,1,1,1,4.75,4.75,3.50,4.50
2,21,0,0,0,0,1,0,0,0,0,...,0,2,1,1,1,1,4.50,4.50,3.25,4.50
3,22,0,0,0,0,1,0,0,0,0,...,0,3,1,1,1,1,4.50,4.50,3.75,3.75
4,43,0,0,0,0,1,0,0,0,0,...,0,1,1,1,1,1,4.50,4.25,3.50,4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,32,0,0,0,0,0,0,0,0,0,...,1,2,1,0,0,0,3.25,3.25,3.50,3.25
2012,43,0,0,0,0,0,0,0,0,0,...,1,3,1,0,0,0,1.75,3.75,3.50,1.75
2013,29,0,0,0,0,0,0,0,0,0,...,1,2,1,1,0,0,3.50,3.75,3.75,3.25
2014,37,0,0,0,0,0,0,0,0,0,...,1,2,1,1,1,1,2.75,4.00,3.25,2.50


##ADECUAMOS LOS DATOS PARA INGRESAR AL MODELO

In [13]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_OneHot_results)##MATRIZ EL DATAFRAME NORMALIZADO

##UTILIZAMOS TSNE

In [14]:
TSN = TSNE(n_components=2)  # Especifica el número de componentes principales que deseas obtener
TSN_result = TSN.fit_transform(scaled_data)
TSN_df = pd.DataFrame(data=TSN_result, columns=['PC1', 'PC2'])
# plt.scatter(TSN_df['PC1'], TSN_df['PC2'])
# plt.xlabel('Componente Principal 1')
# plt.ylabel('Componente Principal 2')
# plt.title('TSNE')
# plt.show()

c:\Users\juans\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\juans\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


##CARGAMOS EL MODELO

In [15]:
modelo_kmeans = joblib.load('Modelos/modelo_kmeans.pkl')

c:\Users\juans\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


##REALIZAMOS LA INFERENCIA

In [16]:
predicciones = modelo_kmeans.predict(TSN_df)

In [17]:
predicciones

array([3, 3, 3, ..., 4, 4, 4])

##GENERAMOS EL DATAFRAME CATEGORIZADO EN LOS CLUSTERS

In [18]:
df_original=pd.concat([df.iloc[:,2:16], df_filter_values], axis=1) ##SELECCIONAMOS TODAS LAS FILAS ## ELIMINAMOS LA FECHA Y EL CODIGO
df_original['Cluster']=predicciones
df_original

,1. Nombre de la institución,2. Tipo de Institución,3. Denominación de los servicios instituciones en salud,4. Área de trabajo.,5. Edad (años cumplidos),6. Sexo,7. Estado civil,8. Escolaridad,9. Termino de contratación,10. Tipo de contrato,11. Horas laboradas cada día en la institución,12. Tiempo Laborado en la institución,13. Tiempo Laborado en su actual área o servicio,14. Tiempo laborado en su actual cargo,LIDERAZGO,MOTIVACIÓN,RECIPROCIDAD,PARTICIPACIÓN,Cluster
0,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,30,Mujer,Casado,Técnico,Término indefinido,Prestación de servicios,1–4 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,3.75,4.00,4.00,3.25,3
1,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,27,Hombre,Soltero,Universitaria,Término indefinido,Por la institución,9-12 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,4.75,4.75,3.50,4.50,3
2,Centro médico laboral,Privada,Baja Complejidad,Asistencial ambulatorio,21,Mujer,Soltero,Técnico,Término indefinido,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,4.50,4.50,3.25,4.50,3
3,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,22,Mujer,Soltero,Universitaria,Término indefinido,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,4.50,4.50,3.75,3.75,3
4,Centro médico laboral,Privada,Baja Complejidad,Apoyo operativo,43,Mujer,Soltero,Secundaria,Término fijo,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,4.50,4.25,3.50,4.50,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial ambulatorio,32,Hombre,Unión Libre,Técnico,Prestación de servicios,Por Cooperativa o asociación,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año,3.25,3.25,3.50,3.25,4
2012,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial hospitalario,43,Mujer,Soltero,Universitaria,Término fijo,Por la institución,5-8 horas,Menos de 1 año,Menos de 1 año,Menos de 1 año,1.75,3.75,3.50,1.75,4
2013,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Apoyo operativo,29,Mujer,Soltero,Técnico,Término fijo,Por Cooperativa o asociación,5-8 horas,De 1 a 5 años,Menos de 1 año,Menos de 1 año,3.50,3.75,3.75,3.25,4
2014,VIVA 1A IPS LAURELES,Privada,Mediana complejidad,Asistencial hospitalario,37,Hombre,Unión Libre,Técnico,Término fijo,Por la institución,5-8 horas,De 1 a 5 años,De 1 a 5 años,De 1 a 5 años,2.75,4.00,3.25,2.50,4


GUARDAMOS LOS RESULTADOS

In [19]:
df_original.to_excel('ResultadosAnalisisCluster/ResultadosClusters.xlsx', index=False)